<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_4/blob/main/Session_4_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [5]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 74.9MB/s]


In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [7]:
print(vars(train_data.examples[0]))

{'text': ['Dressed', 'to', 'Kill', '(', '1980', ')', 'is', 'a', 'mystery', 'horror', 'film', 'from', 'Brian', 'De', 'Palma', 'and', 'it', 'really', 'works', '.', 'The', 'atmosphere', 'is', 'right', 'there', '.', 'The', 'atmosphere', 'that', 'makes', 'you', 'scared', '.', 'And', 'is', "n't", 'that', 'what', 'a', 'horror', 'film', 'is', 'supposed', 'to', 'do', '.', 'All', 'the', 'actors', 'are', 'in', 'the', 'right', 'places', '.', 'Michael', 'Caine', 'is', 'perfect', 'as', 'Dr.', 'Robert', 'Elliott', ',', 'the', 'shrink', 'with', 'a', 'little', 'secret', '.', 'Angie', 'Dickinson', 'as', 'Kate', 'Miller', ',', 'the', 'sexually', 'frustrated', 'mature', 'woman', 'is', 'terrific', '.', 'Keith', 'Gordon', 'as', 'her', 'son', 'Peter', 'is', 'brilliant', '.', 'Nancy', 'Allen', 'as', 'Liz', 'Blake', 'the', 'call', 'girl', 'is', 'fantastic', '.', 'Dennis', 'Franz', 'does', 'his', 'typical', 'detective', 'role', '.', 'His', 'Detective', 'Marino', 'is', 'one', 'of', 'the', 'most', 'colorful', 'in

In [8]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [10]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [11]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [12]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202085), (',', 192344), ('.', 165350), ('and', 109048), ('a', 109006), ('of', 100263), ('to', 93393), ('is', 76161), ('in', 60995), ('I', 54145), ('it', 53577), ('that', 49117), ('"', 44429), ("'s", 43553), ('this', 42185), ('-', 37157), ('/><br', 35889), ('was', 35031), ('as', 30319), ('with', 29691)]


In [13]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [14]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [56]:
BATCH_SIZE = 25

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [57]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,n_layers=1):
        
        super(LSTM, self).__init__()
        self.output_size = output_dim
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,n_layers)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, hidden):
        #batch_size = text.size(0)

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        lstm_out, hidden = self.lstm(embedded, hidden)
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        #assert torch.equal(lstm_out[-1,:,:], hidden.squeeze(0))
        
        return self.fc(lstm_out[-1,:,:])
  
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden  

In [58]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [59]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,867,049 trainable parameters


In [60]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [61]:
criterion = nn.BCEWithLogitsLoss()

In [62]:
model = model.to(device)
criterion = criterion.to(device)

In [63]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [64]:
def train(model, iterator, optimizer, criterion,h):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        # h = model.init_hidden(batch.text.size(0))
        h = tuple([e.data for e in h])
        
        optimizer.zero_grad()
                
        predictions = model(batch.text,h).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [65]:
def evaluate(model, iterator, criterion,h):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            # h = model.init_hidden(batch.text.size(0))
            h = tuple([e.data for e in h])

            predictions = model(batch.text,h).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [66]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [67]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    h = model.init_hidden(BATCH_SIZE)
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion,h)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion,h)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 35s
	Train Loss: 0.693 | Train Acc: 49.87%
	 Val. Loss: 0.693 |  Val. Acc: 50.52%
Epoch: 02 | Epoch Time: 0m 35s
	Train Loss: 0.693 | Train Acc: 50.39%
	 Val. Loss: 0.693 |  Val. Acc: 50.57%
Epoch: 03 | Epoch Time: 0m 35s
	Train Loss: 0.693 | Train Acc: 50.10%
	 Val. Loss: 0.693 |  Val. Acc: 50.51%
Epoch: 04 | Epoch Time: 0m 35s
	Train Loss: 0.693 | Train Acc: 50.30%
	 Val. Loss: 0.693 |  Val. Acc: 50.36%
Epoch: 05 | Epoch Time: 0m 36s
	Train Loss: 0.693 | Train Acc: 50.45%
	 Val. Loss: 0.693 |  Val. Acc: 50.52%


In [69]:
model.load_state_dict(torch.load('tut1-model.pt'))
h = model.init_hidden(BATCH_SIZE)
test_loss, test_acc = evaluate(model, test_iterator, criterion,h)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.692 | Test Acc: 52.90%
